# Lab 2

In [ ]:
import numpy
from IPython.display import clear_output
from time import sleep
from random import randint, choice

In [ ]:
OPEN = 0
PLAYER_1 = 1
PLAYER_2 = 2

In [ ]:
DELAY_TIME = 1 # seconds

#### Function to Initialize board

In [ ]:
def create_board():
	return numpy.zeros((6, 7))

#### Function to print board

In [ ]:
def print_board(board: numpy.ndarray):
    clear_output()

    flipped_board = numpy.flip(board, 0)

    OPEN_SPACE_SYMBOL = "○"
    COIN_SYMBOL = "●"

    output = ""

    for index, element in numpy.ndenumerate(flipped_board):
        if element == PLAYER_1:
            output += "\x1b[31m" + COIN_SYMBOL + "\x1b[0m "
        elif element == PLAYER_2:
            output += "\x1b[33m" + COIN_SYMBOL + "\x1b[0m "
        elif element == OPEN:
            output += OPEN_SPACE_SYMBOL + " "
        if index[1] == 6:
            output += "\n"

    print(output + "-------------\n0 1 2 3 4 5 6")
    sleep(DELAY_TIME)

#### Function to check if a move is valid

In [ ]:
def valid_column(board: numpy.ndarray, column: int,):
    return column >= 0 and column <= 6 and board[5][column] == OPEN

#### Function that returns a list of valid moves

In [ ]:
def list_valid_columns(board: numpy.ndarray):
	valid_cols = []
	for col in range(7):
		if valid_column(board, col):
			valid_cols.append(col)
	return valid_cols

#### Function to check which row a move will be placed in if valid

In [ ]:
def open_row(board: numpy.ndarray, column: int):
	for i in range(6):
		if board[i][column] == OPEN:
			return i

#### Function to make a move

In [ ]:
def drop_coin(board: numpy.ndarray, row: int, column: int, coin: int):
	board[row][column] = coin

#### Function to check if a player has won

In [ ]:
def winning_move(board: numpy.ndarray, player: int):
	# Check horizontal locations for win
	for c in range(4):
		for r in range(6):
			if board[r][c] == player and board[r][c+1] == player and board[r][c+2] == player and board[r][c+3] == player:
				return True

	# Check vertical locations for win
	for c in range(7):
		for r in range(3):
			if board[r][c] == player and board[r+1][c] == player and board[r+2][c] == player and board[r+3][c] == player:
				return True

	# Check positively sloped diaganols
	for c in range(4):
		for r in range(3):
			if board[r][c] == player and board[r+1][c+1] == player and board[r+2][c+2] == player and board[r+3][c+3] == player:
				return True

	# Check negatively sloped diaganols
	for c in range(4):
		for r in range(3, 6):
			if board[r][c] == player and board[r-1][c+1] == player and board[r-2][c+2] == player and board[r-3][c+3] == player:
				return True

#### Function to check if game is over

In [ ]:
def terminal_node(board: numpy.ndarray):
    if winning_move(board, PLAYER_1) or winning_move(board, PLAYER_2):
        return True
    elif len(list_valid_columns(board)) == 0:
        return True
    else:
        for row in range(6):
            for col in range(7):
                if board[row][col] == OPEN:
                    return False
        return True

#### Function to evaluate each subsection of the board and return a score

In [ ]:
def evaluate_window(window: list, player: int):
	score = 0
	opponent = PLAYER_1 if player == PLAYER_2 else PLAYER_2

	if window.count(player) == 3 and window.count(OPEN) == 1:
		score += 5
	elif window.count(player) == 2 and window.count(OPEN) == 2:
		score += 2

	if window.count(opponent) == 3 and window.count(OPEN) == 1:
		score -= 4
	
	return score

#### Function to scan the board in different directions and evaluating each subsection

In [ ]:
def score_position(board: numpy.ndarray, player: int):
	score = 0

	# Score center column
	center_array = [int(i) for i in list(board[:, 3])]
	score += center_array.count(player) * 3

	# Score horizontal
	for r in range(6):
		row_array = [int(i) for i in list(board[r, :])]
		for c in range(4):
			score += evaluate_window(row_array[c:c+4], player)

	# Score vertical
	for c in range(7):
		col_array = [int(i) for i in list(board[:, c])]
		for r in range(3):
			evaluate_window(col_array[r:r+4], player)

	# Score positive sloped diagonal
	for r in range(3):
		for c in range(4):
			evaluate_window([board[r+i][c+i] for i in range(4)], player)

	# Score negative sloped diagonal
	for r in range(3, 6):
		for c in range(4):
			evaluate_window([board[r-i][c+i] for i in range(4)], player)

	return score

### Function applying minmax algorithm to find the best move

In [ ]:
def minmax(board: numpy.ndarray, depth: int, alpha: int, beta: int, maximizingPlayer: bool, player: int):
    valid_cols = list_valid_columns(board)
    opponent = PLAYER_1 if player == PLAYER_2 else PLAYER_2

    if terminal_node(board):
        if winning_move(board, player):
            return None, 100
        elif winning_move(board, opponent):
            return None, -100
        else:
            return None, 0
        
    elif depth == 0:
        if maximizingPlayer:
            return None, score_position(board, player)
        else:
            return None, score_position(board, opponent)

    if maximizingPlayer:
        value = -numpy.inf
        column = None
        for col in valid_cols:
            row = open_row(board, col)
            if row is None:
                continue
            temp_board = board.copy()
            drop_coin(temp_board, row, col, player)
            new_score = minmax(temp_board, depth - 1, alpha, beta, False, player)[1]
            if new_score > value:
                value = new_score
                column = col

            alpha = max(alpha, value)
            if alpha >= beta:
                break

        return column, value
    
    else:
        value = numpy.inf
        column = None
        for col in valid_cols:
            row = open_row(board, col)
            if row is None:
                continue
            temp_board = board.copy()
            drop_coin(temp_board, row, col, opponent)
            new_score = minmax(temp_board, depth - 1, alpha, beta, True, player)[1]
            if new_score < value:
                value = new_score
                column = col

            beta = min(beta, value)
            if alpha >= beta:
                break

        return column, value

In [ ]:
game_over = False
board = create_board()
turn = randint(PLAYER_1, PLAYER_2)
winner = None

In [ ]:
print_board(board)

while not game_over:
    if turn == PLAYER_1:
        valid_move = False
        while not valid_move:
            column = int(input("Player 1 move: "))

            if valid_column(board, column):
                drop_coin(board, open_row(board, column), column, PLAYER_1)
                valid_move = True
            else:
                print("Invalid column. Please choose another one.")
                sleep(DELAY_TIME)

        if winning_move(board, PLAYER_1):
            winner = PLAYER_1
            game_over = True
    elif turn == PLAYER_2:
        column = minmax(board, 5, -numpy.inf, numpy.inf, True, PLAYER_2)[0]

        if valid_column(board, column):
            drop_coin(board, open_row(board, column), column, PLAYER_2)

            if winning_move(board, PLAYER_2):
                winner = PLAYER_2
                game_over = True

    print_board(board)

    if winner is not None:
        print(f"Player {winner} wins!")

    elif len(list_valid_columns(board)) == 0:
        print("Game Over. It's a draw.")
        game_over = True

    turn = PLAYER_2 if turn == PLAYER_1 else PLAYER_1
